In [21]:
import torch
import pandas as pd
import sys

In [22]:
sys.path.append('../src')
from preprocess import initialize_clip_model, generate_embedding
from retrieval import *

In [23]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"
print(f"Device is {device}")

Device is mps


In [24]:
df = pd.read_csv('benchmark.csv')
df.head

<bound method NDFrame.head of                                                   Pid  \
0   230765.156074.66EA81FE0710E584.26DCA022C55D334...   
1   230765.156074.66EA81FE0710E584.26DCA022C55D334...   
2   230765.156074.66EA81FE0710E584.26DCA022C55D334...   
3   178866.156074.820F1205554371C6.94435B3E5252BCD...   
4   178866.156074.820F1205554371C6.94435B3E5252BCD...   
5   178866.156074.820F1205554371C6.94435B3E5252BCD...   
6   178866.156074.820F1205554371C6.BF3F547DC44EC64...   
7   178866.156074.820F1205554371C6.BF3F547DC44EC64...   
8   178866.156074.820F1205554371C6.BF3F547DC44EC64...   
9   159390.1.5EDD.B3A17A41A080BDA.US-10292524WH-5-...   
10  159390.1.5EDD.B3A17A41A080BDA.US-10292524WH-5-...   
11  159390.1.5EDD.B3A17A41A080BDA.US-10292524WH-5-...   
12  159390.1.5EDD.7568CF40E1FC370A.US-30284883TO-3...   
13  159390.1.5EDD.7568CF40E1FC370A.US-30284883TO-3...   
14  159390.1.5EDD.7568CF40E1FC370A.US-30284883TO-3...   
15  230765.156074.9E8AD04DF5D4FE7B.3BC8878B87A0460...   
1

In [25]:
# Models to test
clip_model = "openai/clip-vit-base-patch32"
initialize_clip_model(clip_model)

(CLIPProcessor:
 - image_processor: CLIPImageProcessor {
   "crop_size": {
     "height": 224,
     "width": 224
   },
   "do_center_crop": true,
   "do_convert_rgb": true,
   "do_normalize": true,
   "do_rescale": true,
   "do_resize": true,
   "image_mean": [
     0.48145466,
     0.4578275,
     0.40821073
   ],
   "image_processor_type": "CLIPImageProcessor",
   "image_std": [
     0.26862954,
     0.26130258,
     0.27577711
   ],
   "resample": 3,
   "rescale_factor": 0.00392156862745098,
   "size": {
     "shortest_edge": 224
   }
 }
 
 - tokenizer: CLIPTokenizerFast(name_or_path='openai/clip-vit-base-patch32', vocab_size=49408, model_max_length=77, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
 	49406: AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_wo

In [26]:
k = 5
hits = 0

for idx, row in df.iterrows():
    query_text = row.get('Query', None)

    embedding = generate_embedding(query_text=query_text)
    
    # Run retrieval
    retrieved_pids, _ = hybrid_search(query_text, embedding, top_k=k)
    
    # Check if the ground truth Pid is in the top-k results
    if row['Pid'] in retrieved_pids:
        hits += 1

recall_at_k = hits / len(df)
print(f"Recall@{k}: {recall_at_k:.4f}")

Recall@5: 0.7000
